In [46]:
import sys
sys.path.append('../')

import torch
from models.yolo import ZoominHead,MultiHead,Detect, Model
from models.experimental import attempt_load
from utils.general import check_version
import copy
from models.tf import TFModel,TFDetect,TFConv2d
import keras
import tensorflow as tf
# from model.tf import TFD
import numpy as np
from pathlib import Path
import subprocess
import re

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [47]:
yolo_model = attempt_load('../runs/train/prune-66-finetune-carpart-add-car-v2-anchor-tune-3-od-od/weights/best.pt',fuse=True)
tfmodel = TFModel(cfg=yolo_model.yaml,model=yolo_model,nc=yolo_model.nc,imgsz=(320,320))

# print(tfmodel.model.layers[24].m[0].conv.get_weights())

im = tf.zeros((1,320,320 , 3))
b,s = tfmodel.predict(im)
print(b.shape,s.shape)

Fusing layers... 
YOLOv5s-anchortune-3 summary: 157 layers, 6040633 parameters, 0 gradients, 13.1 GFLOPs

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  mo

conv :  ListWrapper([25, 3, 6, 6])
conv :  ListWrapper([55, 25, 3, 3])
conv :  ListWrapper([24, 55, 1, 1])
conv :  ListWrapper([27, 55, 1, 1])
conv :  ListWrapper([40, 51, 1, 1])
conv :  ListWrapper([32, 24, 1, 1])
conv :  ListWrapper([24, 32, 3, 3])
conv :  ListWrapper([117, 40, 3, 3])
conv :  ListWrapper([58, 117, 1, 1])
conv :  ListWrapper([60, 117, 1, 1])
conv :  ListWrapper([110, 118, 1, 1])
conv :  ListWrapper([64, 58, 1, 1])
conv :  ListWrapper([58, 64, 3, 3])
conv :  ListWrapper([64, 58, 1, 1])
conv :  ListWrapper([58, 64, 3, 3])
conv :  ListWrapper([255, 110, 3, 3])
conv :  ListWrapper([127, 255, 1, 1])
conv :  ListWrapper([127, 255, 1, 1])
conv :  ListWrapper([252, 254, 1, 1])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([512, 252, 3, 3])
conv :  ListWrapper([254, 512, 1, 1])


 24      [17, 20, 23]  1     29667  models.yolo.Detect                      [28, [[16, 30], [62, 45], [156, 198]], [128, 256, 512], (320, 320)]


debug conv2d  128 33 torch.Size([33, 85, 1, 1])
conv2d :  [33, 85, 1, 1]
debug conv2d  256 33 torch.Size([33, 154, 1, 1])
conv2d :  [33, 154, 1, 1]
debug conv2d  512 33 torch.Size([33, 229, 1, 1])
conv2d :  [33, 229, 1, 1]
(<tf.Tensor: shape=(1, 2100, 33), dtype=float32, numpy=
array([[[   0.014723,    0.014237,    0.038613, ...,    0.013105,    0.021042,    0.018722],
        [   0.038183,    0.014657,    0.075949, ...,    0.012904,    0.021552,    0.022214],
        [   0.059945,    0.013046,    0.096489, ...,    0.010223,    0.026915,    0.030564],
        ...,
        [    0.75713,     0.92997,     0.61608, ...,    0.050578,    0.044582,    0.077545],
        [    0.85769,     0.92934,     0.53476, ...,     0.04519,    0.038642,    0.072902],
        [    0.94518,     0.93003,     0.48559, ...,    0.042337,    0.037552,    0.076601]]], dtype=float32)>,) (1, 2100, 33)
(1, 2100, 4) (1, 2100, 28)


In [8]:
ins = [tf.ones((1,40,40 , 85)),tf.ones((1,20,20 , 154)),tf.ones((1,10,10 , 229))]
tf.math.reduce_sum(detect(ins))

<tf.Tensor: shape=(), dtype=float32, numpy=2931.0583>

In [9]:
detect.ch = [85,154,229]
print(detect.ch)

ListWrapper([85, 154, 229])


In [49]:
class TFZoominHead(keras.layers.Layer):
    def __init__(self,torch_module):
        super().__init__()
        with torch.no_grad():
            c2,c1,k = torch_module.conv.weight.shape[:-1]
            self.conv = TFConv2d(c1,c2,k,bias=True,w=torch_module.conv)
            self.act_conv = lambda x: keras.activations.swish(x)
            self.pool = keras.layers.AveragePooling2D(2)
            self.flatten = keras.layers.Flatten()
            self.ln1 = keras.layers.Dense(25,use_bias=True,
                            kernel_initializer=keras.initializers.Constant(torch_module.ln1.weight.permute(1,0).numpy()),
                            bias_initializer=keras.initializers.Constant(torch_module.ln1.bias.numpy()))
        
            self.act1 = lambda x: keras.activations.swish(x)
            self.ln2 = keras.layers.Dense(2,use_bias=True,
                                    kernel_initializer=keras.initializers.Constant(torch_module.ln2.weight.permute(1,0).numpy()),
                                    bias_initializer=keras.initializers.Constant(torch_module.ln2.bias.numpy()))
            self.softmax = keras.layers.Softmax()
            
    def call(self,inputs):
        inputs = self.act_conv(self.conv(inputs))
        inputs = self.pool(inputs)
        inputs = self.flatten(inputs)
        
        inputs = self.ln2(self.act1(self.ln1(inputs)))
        
        inputs = self.softmax(inputs)[:,0]
        
        return inputs
    
class TFMuliHead(TFDetect):
    # YOLOv5 Segment head for segmentation models
    def __init__(self, tfdetect_module,zoomin_module):
        super().__init__(tfdetect_module.nc, tfdetect_module.anchors, tfdetect_module.ch, tfdetect_module.imgsz)
        self.m = tfdetect_module.m
        self.detect = TFDetect.call
        self.stride = tfdetect_module.stride
        self.nl = tfdetect_module.nl
        self.na = tfdetect_module.na
        self.grid = tfdetect_module.grid
        self.anchors = tfdetect_module.anchors
        self.anchor_grid = tfdetect_module.anchor_grid
        
        self.i = tfdetect_module.i
        self.f = tfdetect_module.f
        self.t = str(type(self))[8:-2].replace('__main__.','')
#         print(self.t,self.i,self.f)
        
        self.zoomin_head = zoomin_module
        

    def call(self, x):
        z = self.zoomin_head(x[-1])
        x = self.detect(self, x)
        
        return (x[0],z)

In [50]:
# detect module
detect = tfmodel.model.layers[24]
detect.ch = [85,154,229]

# zoomin module
ckpt = torch.load('zoomin_model.pth')
torch_zoomin = ZoominHead()
torch_zoomin.load_state_dict(ckpt)

tf_zoomin = TFZoominHead(torch_zoomin)


mh = TFMuliHead(detect,tf_zoomin)
new_sequantail = tfmodel.model.layers[:-1]
new_sequantail.append(mh)
tfmodel.model = keras.Sequential(new_sequantail)

debug conv2d  229 1 torch.Size([1, 229, 1, 1])
conv2d :  [1, 229, 1, 1]


In [51]:
im = tf.zeros((1,320,320 , 3))
b,s,zoomin = tfmodel.predict(im)
print(b.shape,s.shape,zoomin)

(<tf.Tensor: shape=(1, 2100, 33), dtype=float32, numpy=
array([[[   0.014723,    0.014237,    0.038613, ...,    0.013105,    0.021042,    0.018722],
        [   0.038183,    0.014657,    0.075949, ...,    0.012904,    0.021552,    0.022214],
        [   0.059945,    0.013046,    0.096489, ...,    0.010223,    0.026915,    0.030564],
        ...,
        [    0.75713,     0.92997,     0.61608, ...,    0.050578,    0.044582,    0.077545],
        [    0.85769,     0.92934,     0.53476, ...,     0.04519,    0.038642,    0.072902],
        [    0.94518,     0.93003,     0.48559, ...,    0.042337,    0.037552,    0.076601]]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([  1.438e-06], dtype=float32)>) (1, 2100, 33)
(1, 2100, 4) (1, 2100, 28) tf.Tensor([  1.438e-06], shape=(1,), dtype=float32)


In [36]:
detect = tfmodel.model.layers[24]
print(detect.training)
ins = [tf.ones((1,40,40 , 85)),tf.ones((1,20,20 , 154)),tf.ones((1,10,10 , 229))]
print(detect(ins))

False
(<tf.Tensor: shape=(1, 2100, 33), dtype=float32, numpy=
array([[[   0.012407,    0.010698,    0.057826, ...,   0.0061914,   0.0048229,     0.01183],
        [   0.037407,    0.010698,    0.057826, ...,   0.0061914,   0.0048229,     0.01183],
        [   0.062407,    0.010698,    0.057826, ...,   0.0061914,   0.0048229,     0.01183],
        ...,
        [    0.75088,     0.95161,     0.39843, ...,   0.0028679,   0.0012659,    0.043598],
        [    0.85088,     0.95161,     0.39843, ...,   0.0028679,   0.0012659,    0.043598],
        [    0.95088,     0.95161,     0.39843, ...,   0.0028679,   0.0012659,    0.043598]]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([    0.43998], dtype=float32)>)


In [52]:
batch_size, ch, *imgsz = (1,3,320,320)  # BCHW
print(imgsz)
tf_model = tfmodel
im = tf.zeros((batch_size, *imgsz, ch))  # BHWC order for TensorFlow
# _ = tf_model.predict(im, tf_nms, agnostic_nms, topk_per_class, topk_all, iou_thres, conf_thres)
inputs = tf.keras.Input(shape=(*imgsz, ch), batch_size= batch_size)
outputs = tf_model.predict(inputs)
print('debug shape output : ',outputs)
keras_model = tf.keras.Model(inputs=inputs, outputs=outputs)
keras_model.trainable = False

keras_model.summary()

[320, 320]
(<KerasTensor: shape=(1, 2100, 33) dtype=float32 (created by layer 'tf_muli_head_3')>, <KerasTensor: shape=(1,) dtype=float32 (created by layer 'tf_muli_head_3')>) (1, 2100, 33)
debug shape output :  (<KerasTensor: shape=(1, 2100, 4) dtype=float32 (created by layer 'tf.concat_2')>, <KerasTensor: shape=(1, 2100, 28) dtype=float32 (created by layer 'tf.math.multiply_2')>, <KerasTensor: shape=(1,) dtype=float32 (created by layer 'tf_muli_head_3')>)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(1, 320, 320, 3)]   0           []                               
                                                                                                  
 tf_conv_171 (TFConv)           (1, 160, 160, 25)    2725        ['input_4[0][0]']                
                           

                                                                                                  
 tf.math.subtract_4 (TFOpLambda  (1, 2100, 1)        0           ['tf.split_2[0][1]',             
 )                                                                'tf.math.truediv_8[0][0]']      
                                                                                                  
 tf.math.subtract_5 (TFOpLambda  (1, 2100, 1)        0           ['tf.split_2[0][0]',             
 )                                                                'tf.math.truediv_9[0][0]']      
                                                                                                  
 tf.__operators__.add_4 (TFOpLa  (1, 2100, 1)        0           ['tf.split_2[0][1]',             
 mbda)                                                            'tf.math.truediv_10[0][0]']     
                                                                                                  
 tf.__oper

In [53]:
# convert saved model

f = 'multi_head_saved_model'
tf_nms = False
spec = tf.TensorSpec(keras_model.inputs[0].shape, keras_model.inputs[0].dtype)
m = tf.function(lambda x: keras_model(x))  # full model
m = m.get_concrete_function(spec)
frozen_func = convert_variables_to_constants_v2(m)
tfm = tf.Module()
tfm.__call__ = tf.function(lambda x: frozen_func(x)[:4] if tf_nms else frozen_func(x), [spec])
tfm.__call__(im)
tf.saved_model.save(tfm,
                    f,
                    options=tf.saved_model.SaveOptions(experimental_custom_gradients=False) if check_version(
                        tf.__version__, '2.6') else tf.saved_model.SaveOptions())

2023-12-05 10:54:42.897640: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 3
2023-12-05 10:54:42.902772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19308 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:17:00.0, compute capability: 8.6
2023-12-05 10:54:42.903184: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21716 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:18:00.0, compute capability: 8.6
2023-12-05 10:54:42.903557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 20871 MB memory:  -> device: 2, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


INFO:tensorflow:Assets written to: multi_head_saved_model/assets


In [54]:
# export pb
f = Path('./multi_head.pb')
m = tf.function(lambda x: keras_model(x))  # full model
m = m.get_concrete_function(tf.TensorSpec(keras_model.inputs[0].shape, keras_model.inputs[0].dtype))
frozen_func = convert_variables_to_constants_v2(m)
frozen_func.graph.as_graph_def()
tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir=str(f.parent), name=f.name, as_text=False)



2023-12-05 10:54:45.441998: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 3
2023-12-05 10:54:45.446762: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19308 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:17:00.0, compute capability: 8.6
2023-12-05 10:54:45.447143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21716 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:18:00.0, compute capability: 8.6
2023-12-05 10:54:45.447489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 20871 MB memory:  -> device: 2, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


'./multi_head.pb'

In [55]:
f = 'best_web_prune_066_s_model_int8_320x320_3_anchors_box_score_wo_nms_od_zoomin_head'
# f = str(file).replace('.pt', '_web_prune_066_s_model_int8_320x320_3_anchors_box_score_wo_nms_od')  # js dir
f_pb = './multi_head.pb'  # *.pb path
# f_pb = ''
f_json = f'{f}/model.json'  # *.json path

int8 = True

args = [
    'tensorflowjs_converter',
    '--input_format=tf_frozen_model',
    '--quantize_uint8' if int8 else '',
    '--output_node_names=Identity,Identity_1,Identity_2',
    str(f_pb),
    str(f), ]
subprocess.run([arg for arg in args if arg], check=True)

json = Path(f_json).read_text()
with open(f_json, 'w') as j:  # sort JSON Identity_* in ascending order
    subst = re.sub(
        r'{"outputs": {"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}}}', r'{"outputs": {"Identity": {"name": "Identity"}, '
        r'"Identity_1": {"name": "Identity_1"}, '
        r'"Identity_2": {"name": "Identity_2"}}}', json)
    j.write(subst)

2023-12-05 10:54:55.216379: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-05 10:54:55.216434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-05 10:54:55.216440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missi

Writing weight file best_web_prune_066_s_model_int8_320x320_3_anchors_box_score_wo_nms_od_zoomin_head/model.json...


In [43]:
! mv web_model best_web_prune_066_s_model_int8_320x320_3_anchors_box_score_wo_nms_od_zoomin_head

In [42]:
!rm -rf best_web_prune_066_s_model_int8_320x320_3_anchors_box_score_wo_nms_od_zoomin_head